In [1]:
from flask import Flask, jsonify
from flask_cors import CORS
import cv2
import threading

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Global variables to store the video capture object and frame
cap = None
frame = None
lock = threading.Lock()  # Thread lock to protect access to frame


def video_stream():
    global cap, frame
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Unable to access the camera")
        return

    while True:
        ret, current_frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame from camera")
            break

        # Lock the access to frame before updating it
        with lock:
            frame = current_frame

    cap.release()


def calculate_face_distance():
    global frame

    with lock:
        if frame is None:
            return {'error': 'No frame captured'}

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        if len(faces) > 0:
            (x, y, w, h) = faces[0]
            distance = 100.0 / w  # Mock calculation of distance
            return {'distance': distance}
        else:
            return {'error': 'No face detected'}


@app.route('/get_distance', methods=['GET'])
def get_distance():
    result = calculate_face_distance()
    return jsonify(result)


if __name__ == "__main__":
    # Start the webcam stream in a separate thread
    t = threading.Thread(target=video_stream)
    t.daemon = True
    t.start()

    # Run the Flask app
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.7.20.154:5000
Press CTRL+C to quit
10.7.20.154 - - [21/Oct/2024 18:24:08] "GET /get_distance HTTP/1.1" 200 -
10.7.20.154 - - [21/Oct/2024 18:39:55] "GET /get_distance HTTP/1.1" 200 -
